# Evaluating the uw_yolov8 models for SOLAQUA vision videos 

# NTNU_fishdata trained, SOLAQUA tested

### Eval


In [1]:
from ultralytics import YOLO
import subprocess

model = YOLO("../runs_uwyolo/fishdata_adamw150_2gpu/weights/best.pt")

results = model.predict(
    source="../../solaqua/data/exports/vision/2024-08-20_17-14-36/raw/2024-08-20_17-14-36_vision.mp4",   
    imgsz=640,
    conf=0.25,
    iou=0.3,
    save=True,
    project="../runs_uwyolo",
    name="EVAL_fishdata_on_solaqua",
    exist_ok=True,
    save_txt=False,
    save_conf=False
)

# results is a list; use the first element
save_dir = results[0].save_dir
print("✅ Inference done. Results saved in:", save_dir)





WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1242) /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/notebooks/../../solaqua/data/exports/vision/2024-08-20_17-14-36/raw/2024-08-20_17-14-36_vision.mp4: 384x640 (no detections), 49.4ms
video 1/1 (frame 2/1242) /cluster/home/henrban/SOLAQUA-UOD/uw_yolov8/notebooks/../../solaqua/data/exports/vision/2024-08-20_17-14-36/raw/2024-08-20_17-14-36_vision.mp4: 384x640 (no detections), 11.0ms
video 1/1 (frame 3/1242) /cluster/home/henrban/SOLAQUA

### .avi -> mp4

In [3]:
import cv2
from pathlib import Path

# ==== CONFIG ====
input_path = Path("../runs_uwyolo/EVAL_fishdata_on_solaqua/2024-08-20_17-14-36_vision.mp4")

output_path = input_path.with_suffix(".mp4")   # output with same name but .mp4

# ==== OPEN INPUT ====
cap = cv2.VideoCapture(str(input_path))
if not cap.isOpened():
    raise IOError(f"❌ Cannot open video: {input_path}")

fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")   # or 'avc1' for H.264 codec

# ==== OPEN OUTPUT ====
out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

print(f"🎞️ Converting {input_path.name} → {output_path.name} ({width}x{height} @ {fps:.2f}fps) ...")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    out.write(frame)

cap.release()
out.release()

print(f"✅ Done! Saved to: {output_path.resolve()}")


OSError: ❌ Cannot open video: ../runs_uwyolo/EVAL_fishdata_on_solaqua/2024-08-20_17-14-36_vision.mp4

## Annotat images from folder (to use in sbs video)

In [1]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from typing import Union, Optional, Sequence
from pathlib import Path
import cv2
import numpy as np
from ultralytics import YOLO
import glob
from tqdm import tqdm

bag_stem = "2024-08-20_17-14-36"
vision_frames = f"../../solaqua/data/exports/vision/{bag_stem}/raw/image__compressed_image__data"
out_annot    = f"../../solaqua/data/exports/vision/{bag_stem}/predictions"



def annotate_images_from_folder(
    images_dir: Union[str, Path],
    out_dir: Union[str, Path],
    weights: Union[str, Path],
    *,
    imgsz: int = 640,
    conf: float = 0.25,
    iou: float = 0.7,
    device: Optional[str] = None,         # e.g., "0" for GPU, "cpu" for CPU, None = auto
    classes: Optional[Sequence[int]] = None,  # e.g., [0,1] to restrict classes
    pattern: str = "*.jpg",               # change to "*.png" or "*.jpg" as needed
) -> None:
    """
    Run YOLO predictions image-by-image and save annotated images with the same filenames.
    """
    images_dir = Path(images_dir)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Load model once
    model = YOLO(str(weights))
    names = model.names  # can be list/tuple or dict

    # Collect images
    paths = sorted(images_dir.glob(pattern))
    if not paths:
        print(f"No images found in {images_dir} matching {pattern}")
        return

    # Simple deterministic color palette per class
    def cls_color(c: int) -> tuple:
        # nice distinct-ish palette
        palette = [
            (255,  56,  56), (255, 159,  56), (255, 255,  56),
            ( 56, 255,  56), ( 56, 255, 255), ( 56,  56, 255),
            (255,  56, 255), (180, 130,  70), ( 80, 175,  76),
        ]
        return palette[c % len(palette)]

    def draw_boxes(img, boxes, labels):
        h, w = img.shape[:2]
        thick = max(2, int(round(0.0025 * (h + w) * 0.5)))
        for (x1, y1, x2, y2), txt, c in boxes:
            color = cls_color(c)
            cv2.rectangle(img, (x1, y1), (x2, y2), color, thick)
            # label bg box
            (tw, th), _ = cv2.getTextSize(txt, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            ytxt = max(0, y1 - 4)
            cv2.rectangle(img, (x1, ytxt - th - 4), (x1 + tw + 6, ytxt), color, -1)
            cv2.putText(img, txt, (x1 + 3, ytxt - 3), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
        return img

    print(f"[INFO] Annotating {len(paths)} images from {images_dir}")
    for p in tqdm(paths, desc="YOLO annotate"):
        # Read image
        img_bgr = cv2.imread(str(p), cv2.IMREAD_COLOR)
        if img_bgr is None:
            tqdm.write(f"⚠️ Could not read {p}")
            continue

        # Predict on a single image
        res = model.predict(
            source=str(p),
            imgsz=imgsz,
            conf=conf,
            device=device,
            classes=classes,
            verbose=False,
            iou=iou,
        )[0]

        # Collect detections
        boxes_to_draw = []
        if res.boxes is not None and len(res.boxes) > 0:
            xyxy = res.boxes.xyxy.cpu().numpy().astype(int)
            cls  = res.boxes.cls.cpu().numpy().astype(int)
            confs = res.boxes.conf.cpu().numpy()
            for (x1, y1, x2, y2), c, score in zip(xyxy, cls, confs):
                # class name lookup (list/tuple or dict)
                cls_name = names[c] if isinstance(names, (list, tuple)) else names.get(c, str(c))
                label = f"{cls_name} {score:.2f}"
                boxes_to_draw.append(((x1, y1, x2, y2), label, c))

        annotated = img_bgr.copy()
        if boxes_to_draw:
            annotated = draw_boxes(annotated, boxes_to_draw, labels=True)

        # Save with same filename into out_dir
        out_path = out_dir / p.name
        cv2.imwrite(str(out_path), annotated)

    print(f"[DONE] Wrote annotated images to: {out_dir.resolve()}")


annotate_images_from_folder(
    images_dir=vision_frames,
    out_dir=out_annot,
    weights="../runs_uwyolo/fishdata_adamw150_2gpu/weights/best.pt",  # <-- trained model
    imgsz=640,
    conf=0.25,  # tweak this
    iou=0.1,    # tweak this also
    pattern="*.jpg",  # adjust if your frames are png
)


[INFO] Annotating 1241 images from ../../solaqua/data/exports/vision/2024-08-20_17-14-36/raw/image__compressed_image__data


YOLO annotate: 100%|██████████| 1241/1241 [00:53<00:00, 23.26it/s]

[DONE] Wrote annotated images to: /cluster/home/henrban/SOLAQUA-UOD/solaqua/data/exports/vision/2024-08-20_17-14-36/predictions
